# Verification and Invoicing

In [1]:
from dataclasses import dataclass
from pathlib import Path
from datetime import date, time, timedelta
import polars as pl

from type_casting.dates import DayName


@dataclass
class DropboxFolder:
    base_path: Path = None

    def __post_init__(self):
        if self.base_path is None:
            self.base_path = Path.home() / "Dropbox"

    @property
    def transfer(self):
        return (
            self.base_path
            / "Container and Transport/Transport Section/Container Movements/Container Transfer.xlsx"
        )

    @property
    def ops(self):
        return (
            self.base_path
            / "! OPERATION SUPPORTING DOCUMENTATION/2025/2025 IPHS operation activity.xlsx"
        )

    @property
    def emr(self):
        return (
            self.base_path
            / "Container and Transport/Container Section/Storage, PTI and Container Cleaning/IPHS - PTI Wash Records.xlsx"
        )

    def get_path(self, relative_path: str):
        return self.base_path / relative_path


# Usage
dropbox = DropboxFolder()
print(dropbox.ops)

/home/garry-mounac/Dropbox/! OPERATION SUPPORTING DOCUMENTATION/2025/2025 IPHS operation activity.xlsx


## Operations Activity

* Handling

In [2]:
handling_activity: pl.LazyFrame = pl.read_excel(
    dropbox.ops, sheet_name="HANDLING ACTIVITY"
).lazy()

handling_activity.collect_schema()

Schema([('DAY', String),
        ('DATE', Date),
        ('VESSEL NAME', String),
        ('SHIPOWNER/OPERATOR', String),
        ('OPERATION TYPE', String),
        ('BRINE (SAUMURE)', Float64),
        ('DRY (Below -30°C)', Float64),
        ('TOTAL TONNAGE', Float64),
        ('Container Brine', Float64),
        ('Container Dry', Float64),
        ('Transhipment Brine', Float64),
        ('Transhipment Dry', Float64),
        ('Simple Unloading Brine', Float64),
        ('Simple Unloading Dry', Float64),
        ('Unloading to CCCS Brine', Float64),
        ('Unloading to CCCS Dry', Float64),
        ('Additional Stevedores', Int64),
        ('Well-to-Well Transfer', Float64),
        ('Overtime Tonnage', Float64),
        ('Extra Men', Int64),
        ('Overtime Value ($)', Float64),
        ('INVOICE VALUE ', Float64),
        ('Number of Stevedores', Int64),
        ('OPEX', Float64),
        ('OPEX %', Float64),
        ('Comments', String)])

In [4]:
pl.read_excel(dropbox.ops, sheet_name="HANDLING ACTIVITY").filter(
    pl.col("DATE").is_not_null()
).select(pl.col("DAY").cast(DayName).alias("day"), pl.col("DATE").alias("date"),
pl.col("VESSEL NAME").alias("vessel_name"),
pl.col("SHIPOWNER/OPERATOR").alias("customer"),
pl.col("OPERATION TYPE"))

day,date,vessel_name,customer,OPERATION TYPE
enum,date,str,str,str
"""Fri""",2025-01-03,"""ARTZA""","""ATUNSA NV""","""Container Stuffing/ Transhipme…"
"""Fri""",2025-01-03,"""ALAKRANA""","""ECHEBASTAR""","""Container Stuffing"""
"""Sat""",2025-01-04,"""ATERPE ALAI""","""ECHEBASTAR""","""Container Stuffing/ Unloading …"
"""Sat""",2025-01-04,"""ARTZA""","""ATUNSA NV""","""Container Stuffing/ Transhipme…"
"""Sat""",2025-01-04,"""ALAKRANA""","""ECHEBASTAR""","""Container Stuffing"""
…,…,…,…,…
"""Tue""",2025-06-24,"""PLAYA DE ANZORAS""","""BEACH FISHING LIMITED""","""Transhipment"""
"""Wed""",2025-06-25,"""JAI ALAI""","""HARTSWATER LIMITED""","""Transhipment"""
"""Wed""",2025-06-25,"""ELAI ALAI""","""ECHEBASTAR""","""Container Stuffing/ Transhipme…"


In [6]:
pl.read_excel(dropbox.ops, sheet_name="BERTH DUES IPHS")

Month,VESSEL NAME,DATE IN,TIME IN,DATE OUT,TIME OUT,DURATION IN PORT,VALUE ($),Test Column,DISCOUNT,INVOICE VALUE,INVOICING ENTITY,VESSEL TYPE,STATUS,COMMENTS,Total MTD,Berth Dues Refund Eligibility
i64,str,date,datetime[ms],date,datetime[ms],i64,i64,i64,i64,i64,str,str,str,str,str,str
1,"""TXORI BAT""",2025-01-02,1899-12-31 10:20:00,2025-01-04,1899-12-31 16:55:00,3,600,600,null,600,"""HARTSWATER LIMITED""","""SUPPLY BOAT""","""INVOICED""",null,"""""","""N"""
1,"""ARTZA""",2025-01-02,1899-12-31 17:26:00,2025-01-08,1899-12-31 07:20:00,6,8000,8000,-1500,6500,"""ATUNSA NV""","""FISHING VESSEL""","""INVOICED""","""5 days unloading / Last day sh…","""""","""Y"""
1,"""ATERPE ALAI""",2025-01-02,1899-12-31 18:10:00,2025-01-05,1899-12-31 17:00:00,3,3750,3750,-1250,2500,"""ECHEBASTAR FLEET SLU""","""FISHING VESSEL""","""INVOICED""","""2 days unloading / First day s…","""""","""Y"""
1,"""SIERRA LARA""",2025-01-02,1899-12-31 18:34:00,2025-01-06,1899-12-31 07:15:00,4,1000,1000,-100,900,"""SEAWARD CO. LTD""","""CARGO REEFER""","""INVOICED""","""3 days 2nd line / Last day sho…","""""","""N"""
1,"""ALAKRANA""",2025-01-03,1899-12-31 11:05:00,2025-01-10,1899-12-31 07:06:00,7,8750,8750,-1250,7500,"""ECHEBASTAR FLEET SLU""","""FISHING VESSEL""","""INVOICED""","""5 days unloading / 1 day on st…","""""","""Y"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
6,"""PLAYA DE ANZORAS""",2025-06-24,1899-12-31 07:13:00,2025-06-25,1899-12-31 06:35:00,1,1500,1500,null,1500,"""BEACH FISHING LIMITED""","""FISHING VESSEL""",null,null,"""""",null
6,"""BERNICA""",2025-06-23,1899-12-31 14:50:00,null,null,null,0,0,null,0,"""SAPMER""","""FISHING VESSEL""",null,null,"""""",null
6,"""JAI ALAI""",2025-06-23,1899-12-31 06:54:00,null,null,null,0,0,null,0,"""HARTSWATER LIMITED""","""FISHING VESSEL""",null,null,"""""",null


In [7]:
transfer_df: pl.LazyFrame = (
    pl.read_excel(dropbox.transfer, sheet_name="Transfer")
    .lazy()
    .select(
        pl.col("Date").alias("date"),
        pl.col("Container Ref. No.").alias("container_number"),
        pl.col("Line/Client").cast(pl.Categorical).alias("line"),
        pl.col("Movement Type").alias("movement_type"),
        pl.col("Driver").alias("driver"),
        pl.col("From").alias("origin"),
        pl.col("Time out")
        .dt.to_string()
        .str.extract(r"(\d{2}:\d{2}:\d{2})")
        .str.to_time(format="%H:%M:%S")
        .alias("time_out"),
        pl.col("Destination").alias("destination"),
        pl.col("Time in")
        .dt.to_string()
        .str.extract(r"(\d{2}:\d{2}:\d{2})")
        .str.to_time(format="%H:%M:%S")
        .alias("time_in"),
        pl.col("Status").alias("status"),
        pl.col("Type").alias("type"),
        pl.col("Size").alias("size"),
        pl.col("Remarks").alias("remarks"),
    )
)

In [8]:
transfer_df.collect()

date,container_number,line,movement_type,driver,origin,time_out,destination,time_in,status,type,size,remarks
date,str,cat,str,str,str,time,str,time,str,str,str,str
2024-08-09,"""MSFU0002762""","""MAERSK""","""Collection""","""Paul""","""LML""",10:15:00,"""IPHS""",10:30:00,"""Empty""","""Reefer""","""40'""","""Maersk Depot"""
2024-11-23,"""MNBU0298795""","""MAERSK""","""Collection""","""Hubert J""","""LML""",09:16:00,"""IPHS""",09:26:00,"""Empty""","""Reefer""","""40'""","""Maersk Depot"""
2024-11-29,"""TRIU8055553""","""CMA CGM""","""Collection""","""Hunt Deltel (IPHS)""","""LML""",09:44:00,"""IPHS""",09:54:00,"""Empty""","""Reefer""","""40'""","""CMA CGM Depot"""
2024-12-01,"""MMAU1016557""","""MAERSK""","""Collection""","""Land Marine (IPHS)""","""LML""",21:20:00,"""IPHS""",21:29:00,"""Empty""","""Reefer""","""40'""","""Maersk Depot"""
2024-12-01,"""MMAU1031340""","""MAERSK""","""Collection""","""Land Marine (IPHS)""","""LML""",20:00:00,"""IPHS""",20:11:00,"""Empty""","""Reefer""","""40'""","""Maersk Depot"""
…,…,…,…,…,…,…,…,…,…,…,…,…
2025-06-26,"""MNBU3302380""","""MAERSK""","""Collection""","""Paul""","""LML""",10:04:00,"""IPHS""",10:14:00,"""Empty""","""Reefer""","""40'""","""Maersk Depot"""
2025-06-26,"""SUDU8210281""","""MAERSK""","""Collection""","""Paul""","""LML""",11:38:00,"""IPHS""",11:48:00,"""Empty""","""Reefer""","""40'""","""Maersk Depot"""
2025-06-26,"""MNBU4340126""","""MAERSK""","""Collection""","""Paul""","""LML""",13:25:00,"""IPHS""",13:37:00,"""Empty""","""Reefer""","""40'""","""Maersk Depot"""


In [9]:
transfer_with_duration = transfer_df.with_columns(
    duration=pl.when(
        # Check if time_in is between midnight and 7:59 AM
        # AND time_out is later in the day (crossing midnight)
        (pl.col("time_in").is_between(time(0, 0, 0), time(7, 59, 59)))
        .and_(pl.col("time_out").is_between(time(8, 0, 0), time(23, 59, 59)))
        .and_(pl.col("time_in").lt(pl.col("time_out")))
    )
    .then(
        # Add one day to time_in when crossing midnight
        (pl.col("date") + pl.duration(days=1)).dt.combine(pl.col("time_in"))
        - pl.col("date").dt.combine(pl.col("time_out"))
    )
    .otherwise(
        # Normal case: same day calculation
        pl.col("date").dt.combine(pl.col("time_in"))
        - pl.col("date").dt.combine(pl.col("time_out"))
    )
).collect()

In [ ]:
transfer_with_duration.filter(pl.col("duration").lt(pl.duration())).with_columns([
    pl.col("duration").dt.total_seconds().abs().alias("abs_seconds"),
    (pl.col("duration").dt.total_seconds() < 0).alias("is_negative")
]).select(
    pl.col("date"),
    pl.col("container_number"),
    pl.col("movement_type"),
    pl.col("driver"),
    (
        pl.when(pl.col("is_negative")).then(pl.lit("-")).otherwise(pl.lit("")) +
        (pl.col("abs_seconds") // 3600).cast(pl.Int32).cast(pl.String) +
        ":" +
        ((pl.col("abs_seconds") % 3600) // 60).cast(pl.Int32).cast(pl.String).str.pad_start(2, "0")
    ).alias("duration")
)

ColumnNotFoundError: -

Resolved plan until failure:

	---> FAILED HERE RESOLVING 'sink' <---
DF ["date", "container_number", "line", "movement_type", ...]; PROJECT */16 COLUMNS

In [17]:
print("Negative Duration")

transfer_with_duration.filter(pl.col("duration").lt(pl.duration())).select(
    pl.col("date"),
    pl.col("container_number"),
    pl.col("movement_type"),
    pl.col("driver"),
(
        pl.when(pl.col("duration").dt.total_seconds() < 0).then("-").otherwise("") +
        pl.format(
            "{}:{:02d}",
            (pl.col("duration").dt.total_seconds().abs() // 3600).cast(pl.Int32),
            ((pl.col("duration").dt.total_seconds().abs() % 3600) // 60).cast(pl.Int32)
        )
    ).alias("duration")
)


Negative Duration


ValueError: number of placeholders should equal the number of arguments

In [11]:
print("High Duration (above 1h)")

transfer_with_duration.filter(pl.col("duration").gt(pl.duration(minutes=60))).select(
    pl.col("date"),
    pl.col("container_number"),
    pl.col("movement_type"),
    pl.col("driver"),
    pl.col("duration"),
)

High Duration (above 1h)


date,container_number,movement_type,driver,duration
date,str,str,str,duration[μs]
2024-12-01,"""MNBU0649035""","""Collection""","""Land Marine (IPHS)""",2h 10m
2024-12-02,"""MNBU3369315""","""Collection""","""Land Marine (IPHS)""",4h 10m
2025-03-21,"""MNBU3408371""","""Collection""","""Hunt Deltel (IPHS)""",3h 26m
2025-03-21,"""MNBU3127013""","""Collection""","""Hunt Deltel (IPHS)""",2h 59m
2025-04-01,"""TLLU1173780""","""Delivery""","""Marcel""",1h 10m
2025-04-15,"""MNBU4562972""","""Delivery""","""Nicodemus""",1h 10m
2025-04-17,"""MNBU4145330""","""Delivery""","""Hunt Deltel (IPHS)""",1h 10m
2025-04-18,"""MNBU0615415""","""Collection""","""Nicodemus""",1h 10m
2025-06-02,"""MMAU1200559""","""Collection""","""Paul""",3h 10m


In [12]:
transfer_with_duration.filter(pl.col("container_number").eq("MMAU1200559"))

date,container_number,line,movement_type,driver,origin,time_out,destination,time_in,status,type,size,remarks,duration
date,str,cat,str,str,str,time,str,time,str,str,str,str,duration[μs]
2025-06-02,"""MMAU1200559""","""MAERSK""","""Collection""","""Paul""","""LML""",13:03:00,"""IPHS""",16:13:00,"""Empty""","""Reefer""","""40'""","""Maersk Depot""",3h 10m


### Duplicated record

In [ ]:
transfer_df.with_columns(
    idx=pl.col("container_number")
    + pl.col("date").dt.to_string(format="%d%m%Y")
    + pl.col("movement_type")
).filter(pl.col("idx").is_duplicated()).collect()

date,container_number,line,movement_type,driver,origin,time_out,destination,time_in,status,type,size,remarks,idx
date,str,cat,str,str,str,time,str,time,str,str,str,str,str


In [ ]:
# First, let's prepare the data with proper datetime handling
transfer_with_datetime = transfer_df.with_columns(
    [
        # Create start_datetime (time_out combined with date)
        pl.col("date").dt.combine(pl.col("time_out")).alias("start_datetime"),
        # Create end_datetime with midnight crossing logic
        pl.when(pl.col("time_out") > pl.col("time_in"))
        .then(
            # Service crosses midnight - time_in is on the next day
            (pl.col("date") + pl.duration(days=1)).dt.combine(pl.col("time_in"))
        )
        .otherwise(
            # Normal same-day service
            pl.col("date").dt.combine(pl.col("time_in"))
        )
        .alias("end_datetime"),
        # Add a unique service ID for easier tracking
        pl.int_range(pl.len()).alias("service_id"),
    ]
)

transfer_with_datetime.collect()

date,container_number,line,movement_type,driver,origin,time_out,destination,time_in,status,type,size,remarks,start_datetime,end_datetime,service_id
date,str,cat,str,str,str,time,str,time,str,str,str,str,datetime[μs],datetime[μs],i64
2024-08-09,"""MSFU0002762""","""MAERSK""","""Collection""","""Paul""","""LML""",10:15:00,"""IPHS""",10:30:00,"""Empty""","""Reefer""","""40'""","""Maersk Depot""",2024-08-09 10:15:00,2024-08-09 10:30:00,0
2024-11-23,"""MNBU0298795""","""MAERSK""","""Collection""","""Hubert J""","""LML""",09:16:00,"""IPHS""",09:26:00,"""Empty""","""Reefer""","""40'""","""Maersk Depot""",2024-11-23 09:16:00,2024-11-23 09:26:00,1
2024-11-29,"""TRIU8055553""","""CMA CGM""","""Collection""","""Hunt Deltel (IPHS)""","""LML""",09:44:00,"""IPHS""",09:54:00,"""Empty""","""Reefer""","""40'""","""CMA CGM Depot""",2024-11-29 09:44:00,2024-11-29 09:54:00,2
2024-12-01,"""MMAU1016557""","""MAERSK""","""Collection""","""Land Marine (IPHS)""","""LML""",21:20:00,"""IPHS""",21:29:00,"""Empty""","""Reefer""","""40'""","""Maersk Depot""",2024-12-01 21:20:00,2024-12-01 21:29:00,3
2024-12-01,"""MMAU1031340""","""MAERSK""","""Collection""","""Land Marine (IPHS)""","""LML""",20:00:00,"""IPHS""",20:11:00,"""Empty""","""Reefer""","""40'""","""Maersk Depot""",2024-12-01 20:00:00,2024-12-01 20:11:00,4
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2025-06-06,"""MNBU3102078""","""MAERSK""","""Delivery""","""Nicodemus""","""IPHS""",15:21:00,"""LML""",15:31:00,"""Full""","""Reefer""","""40'""","""STO""",2025-06-06 15:21:00,2025-06-06 15:31:00,5504
2025-06-06,"""MNBU3795835""","""MAERSK""","""Delivery""","""Nicodemus""","""IPHS""",16:19:00,"""LML""",16:29:00,"""Full""","""Reefer""","""40'""","""STO""",2025-06-06 16:19:00,2025-06-06 16:29:00,5505
2025-06-06,"""MNBU3699087""","""MAERSK""","""Delivery""","""Paul""","""IPHS""",10:53:00,"""LML""",11:15:00,"""Full""","""Reefer""","""40'""","""STO""",2025-06-06 10:53:00,2025-06-06 11:15:00,5506


### Washing and PTI Data

In [ ]:
pl.read_excel(dropbox.emr).filter(pl.col("Date Plug").dt.year().eq(2025)).with_columns(
    pl.col("Time Plug")
    .dt.to_string()
    .str.extract(r"(\d{2}:\d{2}:\d{2})")
    .str.to_time(format="%H:%M:%S")
).with_columns(
    pl.col("Time plug out")
    .dt.to_string()
    .str.extract(r"(\d{2}:\d{2}:\d{2})")
    .str.to_time(format="%H:%M:%S")
).drop(pl.col("#"), pl.col("Size"), pl.col("Hours")).with_columns(
    duration_on_plug=(
        (pl.col("Date PTI/Unplug").dt.combine(pl.col("Time plug out")))
        - (pl.col("Date Plug").dt.combine(pl.col("Time Plug")))
    ).dt.total_seconds()
    / 3600
)

Date Plug,Time Plug,Container Ref. No.,Set Point,Unit Manufacturer,Line/Client,Date PTI/Unplug,Time plug out,Sticker,Status,Generator,duration_on_plug
date,time,str,i64,str,str,date,time,str,str,str,f64
2025-01-02,09:30:00,"""MMAU1018360""",-25,"""Starcool""","""MAERSK""",2025-01-02,15:10:00,"""N/A""","""FAILED""","""K6""",5.666667
2025-01-02,09:30:00,"""MNBU0096855""",-25,"""Carrier""","""MAERSK""",2025-01-02,15:10:00,"""PASS""","""PASSED""","""K6""",5.666667
2025-01-02,09:30:00,"""MNBU0235328""",-25,"""Carrier""","""MAERSK""",2025-01-02,15:10:00,"""PASS""","""PASSED""","""K6""",5.666667
2025-01-02,09:30:00,"""MNBU0324739""",-25,"""Carrier""","""MAERSK""",2025-01-02,15:10:00,"""PASS""","""PASSED""","""K6""",5.666667
2025-01-02,09:30:00,"""MNBU0325150""",-25,"""Carrier""","""MAERSK""",2025-01-02,15:10:00,"""PASS""","""PASSED""","""K6""",5.666667
…,…,…,…,…,…,…,…,…,…,…,…
2025-05-12,07:49:00,"""MNBU4562289""",-25,"""Starcool""","""MAERSK""",2025-05-12,10:29:00,"""PASS""","""PASSED""","""AKSA""",2.666667
2025-05-12,07:49:00,"""MNBU9010174""",-25,"""Starcool""","""MAERSK""",2025-05-12,10:29:00,"""PASS""","""PASSED""","""AKSA""",2.666667
2025-05-12,07:49:00,"""MNBU9044359""",-25,"""Starcool""","""MAERSK""",2025-05-12,10:29:00,"""PASS""","""PASSED""","""AKSA""",2.666667


In [9]:
from dataframe.emr import washing

In [15]:
wash_df = await washing()

In [17]:
wash_df.collect()

date,container_number,invoice_to,service_remarks,price
str,enum,enum,str,i64
"""03/01/2024""","""SUDU6106112""","""ECHEBASTAR""","""Unstuffed at CCCS""",30
"""03/01/2024""","""TTNU8054125""","""IOT""","""Clean""",30
"""04/01/2024""","""MMAU1082543""","""MAERSKLINE""","""Clean""",30
"""04/01/2024""","""MMAU1128460""","""MAERSKLINE""","""Clean""",30
"""04/01/2024""","""MNBU3645352""","""MAERSKLINE""","""Clean""",30
…,…,…,…,…
"""27/05/2025""","""MMAU1227170""","""MAERSKLINE""","""ADDED TO THE RECORD""",30
"""29/05/2025""","""MNBU3851604""","""MAERSKLINE""","""ADDED TO THE RECORD""",30
"""30/05/2025""","""MNBU3670822""","""MAERSKLINE""","""ADDED TO THE RECORD""",30
